# <center>Exploratory Data Analysis</center>
***

## 1. Reader's guide
***
This section aims to elaborate on the data we will be handling as well as the end goal of the notebook.

### 1.1 Introduction
<br>
This notebook servers as an Exploratory Data Analysis (EDA) on the live extracted data (courtesy of the <a href="https://dev.twitch.tv/docs/" target="_blank">Twitch</a>) ranging from dates 13/04/2021 until 17/04/2021. This data will be further enriched with statistics for the fastest growing and most followed channels (courtesy of <a href="https://sullygnome.com/" target="_blank">SullyGnome</a>) for the week ranging from 12/04/2021 until 18/04/2021.

### 1.2 Goal of the document
<br>
The goal of this notebook is to find a possible positive (strong or at least moderate) correlation between the different features provided to enable us to recommend the best time slots and streaming categories for the streamer's growth.
<hr>

## 2. Data wrangling  
***  
This section aim to give an overview of the data and clean it up for visualization later. For data manipulation we will be using _pandas_.

### 2.1 Loading the datasets

#### 2.1.1 Live extracted data

Firstly, we need to load in _pandas_ and load all the live extracted files. They are in the form of a _.csv_.

In [1]:
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

To begin with, we will import the _os_ module for file structure traversal. Alongside that, we will be initializing an empty list to store all the ```DataFrame``` objects from the _.csvs_. They will be extracted via the ```read_csv()``` method of _pandas_. The information was extracted from the <a href="https://dev.twitch.tv/" target="_blank">Twitch API</a>. Following that, we will look through all the files, extract the __timestamp__ (date) which is attached to the end of the file name. Then we will be creating the new ```date``` column with the __timestamp__ from the name. Lastly, we will append this ```DataFrame``` to our list.

In [2]:
import os
dfs = []
path = './data/live-extract/'
files = os.listdir(path)
for file in files:
    date = file.split('-')[1].split('.')[0]
    temp_df = pd.read_csv(path + file)
    temp_df['date'] = date
    dfs.append(temp_df)

Now that we have loaded in the datasets as an array of ```DataFrame``` we should distinguish them by the ```date``` since the data was extracted hourly for 4 consecutive days. With that notion out of the way, we will be using the _pandas_ ```MultiIndex``` capabilities to _"label"_ each extraction to its representive date and id of the session. We will also be using the ```set_index()``` method on a ```DataFrame``` to do so.

In [3]:
reindexed_dfs = [dfs[i].set_index(['date', 'id']) for i in range(len(dfs))]

In [4]:
reindexed_dfs[0].head(10)

game_id language            started_at  \
date     id                                                    
04161858 42337364078   509658       de  2021-04-16T15:08:23Z   
         42336640926   509658       en  2021-04-16T12:59:24Z   
         41452204765   509658       de  2021-04-16T16:27:23Z   
         41784969196   509658       de  2021-04-16T16:07:23Z   
         41781849788   509658       fr  2021-04-16T08:15:58Z   
         41451482765   509658       en  2021-04-16T15:14:18Z   
         41450775885   509658       pt  2021-04-16T13:55:37Z   
         42336377022   509658       en  2021-04-16T11:54:12Z   
         41784419244   509658       fr  2021-04-16T15:06:24Z   
         41451382189   509658       es  2021-04-16T15:03:34Z   

                                                                  title  type  \
date     id                                                                     
04161858 42337364078  Geburtstags Special von Repaz! \nSpäter React ...  live   
         42336640926                        #sponsored PlayStation Plus  live   
         41452204765                                               Moin  live   
         41784969196                                #FREiAB18 -- !horde  live   
         41781849788                                LE RéCAP | lestream  live   
         41451482765   Endless Captcha + Scammers Save My Job [Ep. 934]  live   
         41450775885  🟢AO VIVO ROLÊ DE FERRARI COM ROMERO BRITO EM M...  live   
         42336377022                                    𝓵𝓲𝓿𝓮 𝓵𝓪𝓾𝓰𝓱 𝓵𝓸𝓿𝓮  live   
         41784419244           Garde à Vous EP04 | !Commandes !nukekubi  live   
         41451382189  TODOS X EZRA || !info !donación || Youtube Roa...  live   

                        user_id       user_name  viewer_count      user_login  \
date     id                                                                     
04161858 42337364078   64342766         Trymacs         35586         trymacs   
         42336640926   21841789          Nmplol         25858          nmplol   
         41452204765   45044816  MontanaBlack88         23957  montanablack88   
         41784969196   12875057          GRONKH         17903          gronkh   
         41781849788  147337432        lestream         11137        lestream   
         41451482765   32787655         Kitboga          9377         kitboga   
         41450775885  103989988        JonVlogs          9131        jonvlogs   
         42336377022  160504245          39daph          8626          39daph   
         41784419244   26567552  Jirayalecochon          6809  jirayalecochon   
         41451382189  430476278  ElcanaldeJoaco          6658  elcanaldejoaco   

                          game_name  is_mature  
date     id                                     
04161858 42337364078  Just Chatting      False  
         42336640926  Just Chatting      False  
         41452204765  Just Chatting       True  
         41784969196  Just Chatting       True  
         41781849788  Just Chatting      False  
         41451482765  Just Chatting      False  
         41450775885  Just Chatting      False  
         42336377022  Just Chatting      False  
         41784419244  Just Chatting      False  
         41451382189  Just Chatting       True

In [5]:
reindexed_dfs[-1].tail(10)

game_id language            started_at  \
date     id                                                    
04140037 41758304028    75467       en  2021-04-13T20:55:15Z   
         41757955324    75467       fr  2021-04-13T20:11:39Z   
         41416171021    75467       de  2021-04-13T18:44:30Z   
         41418273613    75467       pt  2021-04-13T21:57:29Z   
         42309356606    75467       it  2021-04-13T20:24:35Z   
         41758272716    75467       en  2021-04-13T20:50:58Z   
         42308604446    75467       tr  2021-04-13T19:18:00Z   
         41757356684    75467       pl  2021-04-13T19:12:51Z   
         42309862158    75467       tr  2021-04-13T21:10:46Z   
         42310648254    75467       en  2021-04-13T22:25:36Z   

                                                                  title  type  \
date     id                                                                     
04140037 41758304028       Euro Truck Simulator 2 | ProMods | VTC Miles  live   
         41757955324     (FR)  La T.F.G recrute let's go sur le bitume   live   
         41416171021                              Unterwegs mit der FEZ  live   
         41418273613     boa tarde meu patrão, virei caminhoneiro (18+)  live   
         42309356606                        Andiamo un'po sulle strade   live   
         41758272716                  [UK] Cruising Iberia, ETS2 Style!  live   
         42308604446  taha ile sahura doğru başlamıştır arkadaşlar :...  live   
         41757356684  [ENG/PL] Vtuber!! Hello, Gummy Bears!  Misiaki...  live   
         42309862158                                  Kısıtlamalı yayın  live   
         42310648254  Cruising in Iberia. Drinks and crashes will en...  live   

                        user_id            user_name  viewer_count  \
date     id                                                          
04140037 41758304028  158614780          Mr_L_Gaming             4   
         41757955324  196734247          mathieu2112             4   
         41416171021  164474506       CHRiSplaysLiVE             4   
         41418273613  408512263           ETBrisado_             4   
         42309356606  233301185           gio_86best             4   
         41758272716  594680906  EffeminatelyGeekish             4   
         42308604446  558146621            SFlegendw             4   
         41757356684  173114923            misiaki89             4   
         42309862158   81287344              ayazusa             3   
         42310648254  128015598               nm2588             2   

                               user_login               game_name  
date     id                                                        
04140037 41758304028          mr_l_gaming  Euro Truck Simulator 2  
         41757955324          mathieu2112  Euro Truck Simulator 2  
         41416171021       chrisplayslive  Euro Truck Simulator 2  
         41418273613           etbrisado_  Euro Truck Simulator 2  
         42309356606           gio_86best  Euro Truck Simulator 2  
         41758272716  effeminatelygeekish  Euro Truck Simulator 2  
         42308604446            sflegendw  Euro Truck Simulator 2  
         41757356684            misiaki89  Euro Truck Simulator 2  
         42309862158              ayazusa  Euro Truck Simulator 2  
         42310648254               nm2588  Euro Truck Simulator 2

Now that we have the properly indexed data, we will use the ```concat()``` method on the list of ```DataFrame``` objects to glue them together.

In [6]:
live_df = pd.concat(reindexed_dfs)

In [7]:
live_df.head(10)

game_id language            started_at  \
date     id                                                    
04161858 42337364078   509658       de  2021-04-16T15:08:23Z   
         42336640926   509658       en  2021-04-16T12:59:24Z   
         41452204765   509658       de  2021-04-16T16:27:23Z   
         41784969196   509658       de  2021-04-16T16:07:23Z   
         41781849788   509658       fr  2021-04-16T08:15:58Z   
         41451482765   509658       en  2021-04-16T15:14:18Z   
         41450775885   509658       pt  2021-04-16T13:55:37Z   
         42336377022   509658       en  2021-04-16T11:54:12Z   
         41784419244   509658       fr  2021-04-16T15:06:24Z   
         41451382189   509658       es  2021-04-16T15:03:34Z   

                                                                  title  type  \
date     id                                                                     
04161858 42337364078  Geburtstags Special von Repaz! \nSpäter React ...  live   
         42336640926                        #sponsored PlayStation Plus  live   
         41452204765                                               Moin  live   
         41784969196                                #FREiAB18 -- !horde  live   
         41781849788                                LE RéCAP | lestream  live   
         41451482765   Endless Captcha + Scammers Save My Job [Ep. 934]  live   
         41450775885  🟢AO VIVO ROLÊ DE FERRARI COM ROMERO BRITO EM M...  live   
         42336377022                                    𝓵𝓲𝓿𝓮 𝓵𝓪𝓾𝓰𝓱 𝓵𝓸𝓿𝓮  live   
         41784419244           Garde à Vous EP04 | !Commandes !nukekubi  live   
         41451382189  TODOS X EZRA || !info !donación || Youtube Roa...  live   

                        user_id       user_name  viewer_count      user_login  \
date     id                                                                     
04161858 42337364078   64342766         Trymacs         35586         trymacs   
         42336640926   21841789          Nmplol         25858          nmplol   
         41452204765   45044816  MontanaBlack88         23957  montanablack88   
         41784969196   12875057          GRONKH         17903          gronkh   
         41781849788  147337432        lestream         11137        lestream   
         41451482765   32787655         Kitboga          9377         kitboga   
         41450775885  103989988        JonVlogs          9131        jonvlogs   
         42336377022  160504245          39daph          8626          39daph   
         41784419244   26567552  Jirayalecochon          6809  jirayalecochon   
         41451382189  430476278  ElcanaldeJoaco          6658  elcanaldejoaco   

                          game_name is_mature  
date     id                                    
04161858 42337364078  Just Chatting     False  
         42336640926  Just Chatting     False  
         41452204765  Just Chatting      True  
         41784969196  Just Chatting      True  
         41781849788  Just Chatting     False  
         41451482765  Just Chatting     False  
         41450775885  Just Chatting     False  
         42336377022  Just Chatting     False  
         41784419244  Just Chatting     False  
         41451382189  Just Chatting      True

In [8]:
live_df.tail(10)

game_id language            started_at  \
date     id                                                    
04140037 41758304028    75467       en  2021-04-13T20:55:15Z   
         41757955324    75467       fr  2021-04-13T20:11:39Z   
         41416171021    75467       de  2021-04-13T18:44:30Z   
         41418273613    75467       pt  2021-04-13T21:57:29Z   
         42309356606    75467       it  2021-04-13T20:24:35Z   
         41758272716    75467       en  2021-04-13T20:50:58Z   
         42308604446    75467       tr  2021-04-13T19:18:00Z   
         41757356684    75467       pl  2021-04-13T19:12:51Z   
         42309862158    75467       tr  2021-04-13T21:10:46Z   
         42310648254    75467       en  2021-04-13T22:25:36Z   

                                                                  title  type  \
date     id                                                                     
04140037 41758304028       Euro Truck Simulator 2 | ProMods | VTC Miles  live   
         41757955324     (FR)  La T.F.G recrute let's go sur le bitume   live   
         41416171021                              Unterwegs mit der FEZ  live   
         41418273613     boa tarde meu patrão, virei caminhoneiro (18+)  live   
         42309356606                        Andiamo un'po sulle strade   live   
         41758272716                  [UK] Cruising Iberia, ETS2 Style!  live   
         42308604446  taha ile sahura doğru başlamıştır arkadaşlar :...  live   
         41757356684  [ENG/PL] Vtuber!! Hello, Gummy Bears!  Misiaki...  live   
         42309862158                                  Kısıtlamalı yayın  live   
         42310648254  Cruising in Iberia. Drinks and crashes will en...  live   

                        user_id            user_name  viewer_count  \
date     id                                                          
04140037 41758304028  158614780          Mr_L_Gaming             4   
         41757955324  196734247          mathieu2112             4   
         41416171021  164474506       CHRiSplaysLiVE             4   
         41418273613  408512263           ETBrisado_             4   
         42309356606  233301185           gio_86best             4   
         41758272716  594680906  EffeminatelyGeekish             4   
         42308604446  558146621            SFlegendw             4   
         41757356684  173114923            misiaki89             4   
         42309862158   81287344              ayazusa             3   
         42310648254  128015598               nm2588             2   

                               user_login               game_name is_mature  
date     id                                                                  
04140037 41758304028          mr_l_gaming  Euro Truck Simulator 2       NaN  
         41757955324          mathieu2112  Euro Truck Simulator 2       NaN  
         41416171021       chrisplayslive  Euro Truck Simulator 2       NaN  
         41418273613           etbrisado_  Euro Truck Simulator 2       NaN  
         42309356606           gio_86best  Euro Truck Simulator 2       NaN  
         41758272716  effeminatelygeekish  Euro Truck Simulator 2       NaN  
         42308604446            sflegendw  Euro Truck Simulator 2       NaN  
         41757356684            misiaki89  Euro Truck Simulator 2       NaN  
         42309862158              ayazusa  Euro Truck Simulator 2       NaN  
         42310648254               nm2588  Euro Truck Simulator 2       NaN

With that done we can look at the information present inside this ```DataFrame```.

In [9]:
live_df.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 708265 entries, ('04161858', 42337364078) to ('04140037', 42310648254)
Data columns (total 11 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   game_id       708265 non-null  int64 
 1   language      708265 non-null  object
 2   started_at    708265 non-null  object
 3   title         707866 non-null  object
 4   type          708265 non-null  object
 5   user_id       708265 non-null  int64 
 6   user_name     708265 non-null  object
 7   viewer_count  708265 non-null  int64 
 8   user_login    708265 non-null  object
 9   game_name     708265 non-null  object
 10  is_mature     299352 non-null  object
dtypes: int64(3), object(8)
memory usage: 73.2+ MB


We observe that the ```DataFrame``` consists of __11__ columns and a whopping __708265__ rows. We can also see that there are 2 indexes that we have previously set. They are ```date``` and ```id```.  
<hr>
What we will be doing next is going through a similar process for the other 2 folders related to the <b>fastest growing</b> and <b>most watched</b> 1000 channels for the past week.

#### 2.1.2 Most watched channels

In [10]:
dfs = []
path = './data/7days-most-watched/'
files = os.listdir(path)
for file in files:
    temp_df = pd.read_csv(path + file)
    dfs.append(temp_df)

In [11]:
most_watched_df = pd.concat(dfs)

In [12]:
most_watched_df.head(10)

,Unnamed: 0,Unnamed: 1,Channel,Watch time,Stream time,Peak viewers,Average viewers,Followers,Followers gained,Views gained,Partnered,Mature,Language,Unnamed: 13
0,https://static-cdn.jtvnw.net/jtv_user_pictures...,501,twlevewinsHS,7850130,3165,4100,2480,63634,335,73164,True,False,English,twlevewinsHS
1,https://static-cdn.jtvnw.net/jtv_user_pictures...,502,scumtk,7837020,10080,1933,777,99856,-5817,74085,True,True,English,scumtk
2,https://static-cdn.jtvnw.net/jtv_user_pictures...,503,RTP Arena CS:GO (rtparenacsgo),7817445,2985,8849,2618,155033,2219,520960,True,False,Portuguese,RTP Arena CS:GO (rtparenacsgo)
3,https://static-cdn.jtvnw.net/jtv_user_pictures...,504,Kitboga,7790715,1005,12330,7751,929889,265,123758,True,False,English,Kitboga
4,https://static-cdn.jtvnw.net/jtv_user_pictures...,505,TpaBoMaH,7777800,2040,5770,3812,208323,1740,391756,True,False,Russian,TpaBoMaH
5,https://static-cdn.jtvnw.net/jtv_user_pictures...,506,pimpimenta,7776240,3705,5948,2098,873908,-66,300018,True,False,Portuguese,pimpimenta
6,https://static-cdn.jtvnw.net/jtv_user_pictures...,507,Flight23white,7774365,510,19516,15243,1545964,10069,126938,True,False,English,Flight23white
7,https://static-cdn.jtvnw.net/jtv_user_pictures...,508,NiteNightKid,7747095,2235,7552,3466,35657,214,72005,True,True,English,NiteNightKid
8,https://static-cdn.jtvnw.net/jtv_user_pictures...,509,포셔 (portialyn),7708500,2880,3973,2676,64712,175,114955,True,False,Korean,포셔 (portialyn)
9,https://static-cdn.jtvnw.net/jtv_user_pictures...,510,Pieface23,7701840,1275,11277,6040,480797,3180,88327,True,False,English,Pieface23


In [13]:
most_watched_df.tail(10)

,Unnamed: 0,Unnamed: 1,Channel,Watch time,Stream time,Peak viewers,Average viewers,Followers,Followers gained,Views gained,Partnered,Mature,Language,Unnamed: 13
90,https://static-cdn.jtvnw.net/jtv_user_pictures...,991,RobertoCein,4253775,1695,5954,2509,1392180,9357,144153,True,True,Spanish,RobertoCein
91,https://static-cdn.jtvnw.net/jtv_user_pictures...,992,IKeepItTaco,4252680,2220,2818,1915,264862,-1806,104650,True,False,English,IKeepItTaco
92,https://static-cdn.jtvnw.net/jtv_user_pictures...,993,Harmii,4246800,2115,4653,2007,453615,727,210646,True,False,German,Harmii
93,https://static-cdn.jtvnw.net/jtv_user_pictures...,994,KarasMai,4243740,2625,3157,1616,357409,-2056,102457,True,True,English,KarasMai
94,https://static-cdn.jtvnw.net/jtv_user_pictures...,995,PUBGKorea,4243485,1455,4760,2916,241237,629,131043,True,False,Korean,PUBGKorea
95,https://static-cdn.jtvnw.net/jtv_user_pictures...,996,AgentsRange,4243335,1830,7374,2318,14739,851,983046,True,False,German,AgentsRange
96,https://static-cdn.jtvnw.net/user-default-pict...,997,maskenissen,4242555,345,20590,12297,127676,27987,123653,False,False,English,maskenissen
97,https://static-cdn.jtvnw.net/jtv_user_pictures...,998,명훈 (mlchoins),4240950,2700,3364,1570,92090,881,172692,False,False,Korean,명훈 (mlchoins)
98,https://static-cdn.jtvnw.net/jtv_user_pictures...,999,AnnaCramling,4233345,1665,6969,2542,124348,9445,498801,True,False,English,AnnaCramling
99,https://static-cdn.jtvnw.net/jtv_user_pictures...,1000,DEATHSIE,4225005,3705,2053,1140,33854,714,35575,True,False,English,DEATHSIE


In [14]:
most_watched_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1000 entries, 0 to 99
Data columns (total 14 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Unnamed: 0        1000 non-null   object
 1   Unnamed: 1        1000 non-null   int64 
 2   Channel           1000 non-null   object
 3   Watch time        1000 non-null   int64 
 4   Stream time       1000 non-null   int64 
 5   Peak viewers      1000 non-null   int64 
 6   Average viewers   1000 non-null   int64 
 7   Followers         1000 non-null   int64 
 8   Followers gained  1000 non-null   int64 
 9   Views gained      1000 non-null   int64 
 10  Partnered         1000 non-null   bool  
 11  Mature            1000 non-null   bool  
 12  Language          1000 non-null   object
 13  Unnamed: 13       1000 non-null   object
dtypes: bool(2), int64(8), object(4)
memory usage: 103.5+ KB


<a id='most_watched_id'></a>
We observe that this ```DataFrame``` object consists of <b>14</b> columns and <b>1000</b> rows. The index is the default index set by <em>pandas</em> which is something that we will be taking care of.
<hr>

#### 2.1.3 Fastest growing channels

In [15]:
dfs = []
path = './data/7days-fastest-growing/'
files = os.listdir(path)
for file in files:
    temp_df = pd.read_csv(path + file)
    dfs.append(temp_df)

In [16]:
fastest_growing_df = pd.concat(dfs)

In [17]:
fastest_growing_df.head(10)

,Unnamed: 0,Unnamed: 1,Channel,Watch time,Stream time,Peak viewers,Average viewers,Followers,Followers gained,Views gained,Partnered,Mature,Language,Unnamed: 13
0,https://static-cdn.jtvnw.net/jtv_user_pictures...,401,zin0xfnr,1605,495,9,3,22677,22617,191,False,False,German,zin0xfnr
1,https://static-cdn.jtvnw.net/jtv_user_pictures...,402,GreenAppleTiVi,16185,945,25,17,22986,22595,307,False,False,German,GreenAppleTiVi
2,https://static-cdn.jtvnw.net/jtv_user_pictures...,403,LaWhooligan,36090,810,142,44,23865,22592,2125,False,False,English,LaWhooligan
3,https://static-cdn.jtvnw.net/jtv_user_pictures...,404,V_aoen,8220,885,32,9,23137,22508,307,False,True,English,V_aoen
4,https://static-cdn.jtvnw.net/jtv_user_pictures...,405,zuzk4_aa,2310,450,13,5,22599,22505,121,False,False,Polish,zuzk4_aa
5,https://static-cdn.jtvnw.net/jtv_user_pictures...,406,PapoMC,16622640,1770,25966,9391,1621415,22467,698146,True,True,Spanish,PapoMC
6,https://static-cdn.jtvnw.net/jtv_user_pictures...,407,airestereo,11115,1320,105,8,41468,22387,294,False,False,English,airestereo
7,https://static-cdn.jtvnw.net/jtv_user_pictures...,408,gewoonroy1,8250,1335,18,6,22454,22368,365,False,False,Dutch,gewoonroy1
8,https://static-cdn.jtvnw.net/jtv_user_pictures...,409,elisawavess,3116235,2655,4750,1173,40417,22342,209876,True,True,Spanish,elisawavess
9,https://static-cdn.jtvnw.net/jtv_user_pictures...,410,Czech_KappeR,71955,1710,159,42,32995,22320,7645,False,False,Czech,Czech_KappeR


In [18]:
fastest_growing_df.tail(10)

,Unnamed: 0,Unnamed: 1,Channel,Watch time,Stream time,Peak viewers,Average viewers,Followers,Followers gained,Views gained,Partnered,Mature,Language,Unnamed: 13
90,https://static-cdn.jtvnw.net/jtv_user_pictures...,791,cxltureslol,577650,615,1691,939,83578,13380,57765,False,False,English,cxltureslol
91,https://static-cdn.jtvnw.net/jtv_user_pictures...,792,venomandxneo,43995,3570,34,12,14384,13345,1234,False,True,Turkish,venomandxneo
92,https://static-cdn.jtvnw.net/jtv_user_pictures...,793,mr_barry_allen_,37725,930,105,40,16396,13284,1113,False,False,Russian,mr_barry_allen_
93,https://static-cdn.jtvnw.net/jtv_user_pictures...,794,RobFlexes,285,120,4,2,13543,13281,133,False,False,English,RobFlexes
94,https://static-cdn.jtvnw.net/jtv_user_pictures...,795,CraftingP,1560,315,13,4,13318,13278,188,False,False,German,CraftingP
95,https://static-cdn.jtvnw.net/jtv_user_pictures...,796,goncho,25245345,2730,20703,9247,819427,13234,785241,True,True,Spanish,goncho
96,https://static-cdn.jtvnw.net/jtv_user_pictures...,797,TheKAIRI78,5485710,1110,7855,4942,1105838,13232,120646,True,False,French,TheKAIRI78
97,https://static-cdn.jtvnw.net/jtv_user_pictures...,798,Joe_Bartolozzi,1441365,945,2194,1525,220968,13226,35036,True,False,English,Joe_Bartolozzi
98,https://static-cdn.jtvnw.net/jtv_user_pictures...,799,its_iron,31716885,3780,13106,8390,356973,13203,355585,True,True,English,its_iron
99,https://static-cdn.jtvnw.net/jtv_user_pictures...,800,lucaner_400,1290,570,7,2,13201,13186,119,False,False,German,lucaner_400


In [19]:
fastest_growing_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1000 entries, 0 to 99
Data columns (total 14 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Unnamed: 0        1000 non-null   object
 1   Unnamed: 1        1000 non-null   int64 
 2   Channel           1000 non-null   object
 3   Watch time        1000 non-null   int64 
 4   Stream time       1000 non-null   int64 
 5   Peak viewers      1000 non-null   int64 
 6   Average viewers   1000 non-null   int64 
 7   Followers         1000 non-null   int64 
 8   Followers gained  1000 non-null   int64 
 9   Views gained      1000 non-null   int64 
 10  Partnered         1000 non-null   bool  
 11  Mature            1000 non-null   bool  
 12  Language          1000 non-null   object
 13  Unnamed: 13       1000 non-null   object
dtypes: bool(2), int64(8), object(4)
memory usage: 103.5+ KB


This ```DataFrame``` object has the same qualities as the one <a href="#most_watched_id">preceding it</a>.
<hr>

#### 2.1.4 Most watched games

In [20]:
games_df = pd.read_csv('./data/7days-games/most_watched_games.csv')

In [21]:
games_df.head(10)

,Unnamed: 0,Unnamed: 1,Game,Watch time,Stream time,Peak viewers,Peak channels,Streamers,Average viewers,Average channels,Average viewer ratio,Followers gained,Views gained,FPR,VPR,Unnamed: 15
0,Grand Theft Auto V,1,Grand Theft Auto V,3858308160,36281700,838606,7043,81802,382768,3599,106.34,4050979,63497073,6.699210,105.006777,Grand Theft Auto V
1,Just Chatting,2,Just Chatting,3817617240,50377500,870817,8026,205830,378731,4997,75.78,5644892,99932398,6.723111,119.020275,Just Chatting
2,League of Legends,3,League of Legends,2038034640,51180600,529193,8333,117363,202185,5077,39.82,990206,44384031,1.160838,52.032252,League of Legends
3,Fortnite,4,Fortnite,1233289680,77842680,300531,13805,232235,122350,7722,15.84,6741075,34456004,5.195922,26.558184,Fortnite
4,VALORANT,5,VALORANT,1208156400,47779380,307693,7835,133072,119856,4740,25.29,1060664,33469807,1.331952,42.030441,VALORANT
5,Call of Duty: Warzone,6,Call of Duty: Warzone,1181475060,48264840,278820,8705,100493,117209,4788,24.48,818764,15754868,1.017839,19.585522,Call of Duty: Warzone
6,Minecraft,7,Minecraft,1073379300,43641660,341421,8305,154848,106486,4329,24.60,4028982,16635780,5.539178,22.871421,Minecraft
7,Counter-Strike: Global Offensive,8,Counter-Strike: Global Offensive,1036974120,20340180,362748,4016,62679,102874,2017,50.98,1310455,29275087,3.865615,86.356425,Counter-Strike: Global Offensive
8,Apex Legends,9,Apex Legends,709455180,52156980,120481,8110,126934,70382,5174,13.60,71790,12905807,0.082585,14.846496,Apex Legends
9,Dota 2,10,Dota 2,635300280,9344040,210268,1559,14851,63025,926,67.99,32446,13970781,0.208342,89.709254,Dota 2


In [22]:
games_df.tail(10)

,Unnamed: 0,Unnamed: 1,Game,Watch time,Stream time,Peak viewers,Peak channels,Streamers,Average viewers,Average channels,Average viewer ratio,Followers gained,Views gained,FPR,VPR,Unnamed: 15
90,Special Events,91,Special Events,32497920,417600,82471,378,1559,3224,41,77.82,17841,2156725,2.563362,309.874282,Special Events
91,Clash Royale,92,Clash Royale,32424120,518100,46922,97,1746,3216,51,62.58,57841,754385,6.698437,87.363636,Clash Royale
92,Krunker,93,Krunker,31593180,638760,10014,112,2478,3134,63,49.46,112797,1593172,10.595247,149.649822,Krunker
93,Pummel Party,94,Pummel Party,30725940,522120,50051,225,2947,3048,51,58.85,18920,1655474,2.174213,190.240634,Pummel Party
94,PUBG Mobile,95,PUBG Mobile,30118080,1610580,33329,303,3109,2987,159,18.70,26244,1717403,0.977685,63.979548,PUBG Mobile
95,Core,96,Core,29500080,140700,86293,47,777,2926,13,209.67,10252,637485,4.371855,271.848614,Core
96,Resident Evil 7 biohazard,97,Resident Evil 7 biohazard,29334000,1615800,39747,372,6465,2910,160,18.15,12671,521624,0.470516,19.369625,Resident Evil 7 biohazard
97,Pokémon Trading Card Game,98,Pokémon Trading Card Game,29236620,838080,65024,220,3203,2900,83,34.89,17801,551924,1.274413,39.513459,Pokémon Trading Card Game
98,Crypto,99,Crypto,28466760,777060,9814,110,505,2824,77,36.63,28802,230703,2.223921,17.813528,Crypto
99,Marbles On Stream,100,Marbles On Stream,28456980,897480,16761,185,5350,2823,89,31.71,38353,569510,2.564046,38.073940,Marbles On Stream


In [23]:
games_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 16 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Unnamed: 0            100 non-null    object 
 1   Unnamed: 1            100 non-null    int64  
 2   Game                  100 non-null    object 
 3   Watch time            100 non-null    int64  
 4   Stream time           100 non-null    int64  
 5   Peak viewers          100 non-null    int64  
 6   Peak channels         100 non-null    int64  
 7   Streamers             100 non-null    int64  
 8   Average viewers       100 non-null    int64  
 9   Average channels      100 non-null    int64  
 10  Average viewer ratio  100 non-null    float64
 11  Followers gained      100 non-null    int64  
 12  Views gained          100 non-null    int64  
 13  FPR                   100 non-null    float64
 14  VPR                   100 non-null    float64
 15  Unnamed: 15           10

This `DataFrame` object contains __100__ rows and __16__ columns. The index is defaulted by _pandas_ and is something we will be working on later.

### 2.2 Descriptive analytics and checks

#### 2.2.1 Live extracted data

Firstly, we should make a legend that describes each column. Based on that we can later change the date format or reindex if necessary.  

In [24]:
live_df.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 708265 entries, ('04161858', 42337364078) to ('04140037', 42310648254)
Data columns (total 11 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   game_id       708265 non-null  int64 
 1   language      708265 non-null  object
 2   started_at    708265 non-null  object
 3   title         707866 non-null  object
 4   type          708265 non-null  object
 5   user_id       708265 non-null  int64 
 6   user_name     708265 non-null  object
 7   viewer_count  708265 non-null  int64 
 8   user_login    708265 non-null  object
 9   game_name     708265 non-null  object
 10  is_mature     299352 non-null  object
dtypes: int64(3), object(8)
memory usage: 73.2+ MB


```date``` timestamp of the extracted document of type ```object```
<br>
```id``` session id of type ```int64```
<br>
```game_id``` played game id of type ```int64```
<br>
```language``` the 2 characters language code of type ```object```
<br>
```started_at``` timestamp of the stream start of type ```object```
<br>
```title``` stream's title of type ```object```
<br>
```type``` <em>live</em> or <em>recorded</em> label of type ```object```
<br>
```user_id``` the id of the streamer of type ```int64```
<br>
```user_name``` the streamer's display name of type ```object```
<br>
```viewer_count``` the current viewer count of type ```int64```
<br>
```user_login``` the streamer's username of type ```object```
<br>
```game_name``` the game's name of type ```object```
<br>
```is_mature``` the label whether the stream contains profanity or nudity of type ```object```

We can comfortably drop the `id` index since we are not interested in the session.

In [25]:
live_df.reset_index(level=1, drop=True, inplace=True)

In [26]:
live_df.head(10)

,game_id,language,started_at,title,type,user_id,user_name,viewer_count,user_login,game_name,is_mature
date,,,,,,,,,,,
04161858,509658,de,2021-04-16T15:08:23Z,Geburtstags Special von Repaz! \nSpäter React ...,live,64342766,Trymacs,35586,trymacs,Just Chatting,False
04161858,509658,en,2021-04-16T12:59:24Z,#sponsored PlayStation Plus,live,21841789,Nmplol,25858,nmplol,Just Chatting,False
04161858,509658,de,2021-04-16T16:27:23Z,Moin,live,45044816,MontanaBlack88,23957,montanablack88,Just Chatting,True
04161858,509658,de,2021-04-16T16:07:23Z,#FREiAB18 -- !horde,live,12875057,GRONKH,17903,gronkh,Just Chatting,True
04161858,509658,fr,2021-04-16T08:15:58Z,LE RéCAP | lestream,live,147337432,lestream,11137,lestream,Just Chatting,False
04161858,509658,en,2021-04-16T15:14:18Z,Endless Captcha + Scammers Save My Job [Ep. 934],live,32787655,Kitboga,9377,kitboga,Just Chatting,False
04161858,509658,pt,2021-04-16T13:55:37Z,🟢AO VIVO ROLÊ DE FERRARI COM ROMERO BRITO EM M...,live,103989988,JonVlogs,9131,jonvlogs,Just Chatting,False
04161858,509658,en,2021-04-16T11:54:12Z,𝓵𝓲𝓿𝓮 𝓵𝓪𝓾𝓰𝓱 𝓵𝓸𝓿𝓮,live,160504245,39daph,8626,39daph,Just Chatting,False
04161858,509658,fr,2021-04-16T15:06:24Z,Garde à Vous EP04 | !Commandes !nukekubi,live,26567552,Jirayalecochon,6809,jirayalecochon,Just Chatting,False


What we should do next is sort the values on the `date` index. The `date` is in format `MonthDayHourMinute`. Lastly, we should format it into a more human-readable form.

In [27]:
live_df.reset_index(inplace=True)

In [28]:
live_df['date'] = pd.to_datetime(live_df['date'], format='%m%d%H%M')
live_df['date'] = live_df['date'].dt.strftime('%m-%d-%H:%M')

In [29]:
live_df.sort_values('date', ascending=True, inplace=True)

In [30]:
live_df.set_index(['date'], inplace=True)

In [31]:
live_df.head(10)

,game_id,language,started_at,title,type,user_id,user_name,viewer_count,user_login,game_name,is_mature
date,,,,,,,,,,,
04-13-17:12,490100,ko,2021-04-13T11:37:22Z,로스트아크 서머너 쪼랩 >_<,live,629479892,루하2,10,hye0814,LOST ARK,NaN
04-13-17:12,509667,en,2021-04-13T15:05:56Z,Blog cooking day! Making mozza stuffed chicken...,live,627294963,messinthekitchen,3,messinthekitchen,Food & Drink,NaN
04-13-17:12,509667,en,2021-04-13T13:11:22Z,Cooking Stream,live,204973333,joppevb999,3,joppevb999,Food & Drink,NaN
04-13-17:12,509667,de,2021-04-13T14:51:08Z,Huhn oder Ei? Ich nehm einfach Beides !wasgibt...,live,540687644,liebes_schaefchen,3,liebes_schaefchen,Food & Drink,NaN
04-13-17:12,509667,ko,2021-04-13T14:19:37Z,ㅎ2,live,262433416,셉치,2,gerbong426,Food & Drink,NaN
04-13-17:12,509667,en,2021-04-13T14:38:15Z,Parm. Garlic Monkey Bread (Easy Mode),live,529455629,funthings4kicks,2,funthings4kicks,Food & Drink,NaN
04-13-17:12,509667,en,2021-04-13T14:26:49Z,"Keto Pankytakes, Beet Kvass, Various Nut butte...",live,559291357,DownstreamDinette,2,downstreamdinette,Food & Drink,NaN
04-13-17:12,509667,en,2021-04-12T18:49:14Z,The Justin Wilson Show!,live,182217253,TheJustinWilsonShow,4,thejustinwilsonshow,Food & Drink,NaN
04-13-17:12,509667,it,2021-04-13T08:32:03Z,Caffè e Sigarette,live,660148094,lapiccolasosta,2,lapiccolasosta,Food & Drink,NaN


In [32]:
live_df.tail(10)

,game_id,language,started_at,title,type,user_id,user_name,viewer_count,user_login,game_name,is_mature
date,,,,,,,,,,,
04-17-09:06,27284,en,2021-04-17T03:08:57Z,bad edutainment games,live,8615546,Maghook,8,maghook,Retro,False
04-17-09:06,27284,es,2021-04-17T02:20:54Z,[ES/EN] The Legend of Zelda: Ocarina of Time |...,live,206074070,TamaDrama,8,tamadrama,Retro,False
04-17-09:06,27284,en,2021-04-17T06:25:31Z,Friday! > Now accepting subs:) !prime !merch !...,live,223332994,GrownManRecordNight,8,grownmanrecordnight,Retro,False
04-17-09:06,27284,es,2021-04-17T03:05:47Z,(ESP) Noche de Gotys y wear - PS2 ◕‿◕,live,45617969,Maniatikiller,8,maniatikiller,Retro,True
04-17-09:06,27284,ja,2021-04-17T06:15:16Z,荒川のほとりからスパロボF 東西南北不敗グランドマスターガンダム,live,89926660,サブマリン川口,7,saitouz,Retro,False
04-17-09:06,27284,es,2021-04-17T01:21:18Z,Retro Friday - Castlevania Raceeeee,live,60056516,LotusEaterGs,7,lotuseatergs,Retro,False
04-17-09:06,27284,ko,2021-04-17T04:07:15Z,파엠 각성,live,121484700,최카츄,7,b612no3sai,Retro,False
04-17-09:06,27284,en,2021-04-17T05:49:11Z,Sports Entertainment,live,651917216,blacksaturday1984,7,blacksaturday1984,Retro,True
04-17-09:06,27284,zh,2021-04-17T05:32:51Z,遊戲實況 -隨時都有可能被電話GANK,live,10945182,Screechshout,10,screechshout,Retro,True


Now we can perform the necessary checks for duplicate, null or NAN values.

In [33]:
grouping = live_df.groupby('date') \
    .agg({
            'language': 'nunique',
            'type': 'nunique',
            'user_name': 'nunique',
            'game_name': 'nunique'
        })
grouping

,language,type,user_name,game_name
date,,,,
04-13-17:12,31,1,8226,103
04-13-17:15,31,1,8233,102
04-13-17:22,31,1,8179,102
04-13-18:34,32,1,8442,101
04-13-19:34,32,1,8504,101
04-13-20:35,30,1,8518,101
04-13-21:35,29,1,8500,103
04-13-22:36,31,1,8474,101
04-13-23:36,32,1,8565,101


In [34]:
grouping.describe()

,language,type,user_name,game_name
count,86.000000,86.0,86.000000,86.000000
mean,30.116279,1.0,8235.604651,100.767442
std,2.043173,0.0,514.626084,0.890134
min,25.000000,1.0,7048.000000,100.000000
25%,29.000000,1.0,7961.000000,100.000000
50%,30.500000,1.0,8417.000000,101.000000
75%,32.000000,1.0,8602.500000,101.000000
max,33.000000,1.0,9011.000000,103.000000


In [35]:
grouping[grouping['type'] != 1]['type'].sum()

0

In [36]:
live_df.duplicated().sum()

56702

In [37]:
live_df.isnull().sum()

game_id              0
language             0
started_at           0
title              399
type                 0
user_id              0
user_name            0
viewer_count         0
user_login           0
game_name            0
is_mature       408913
dtype: int64

Based on the observations made above we can conclude that there are around __30__ languages spoken in the streams in the period. All of the channels recorded were __live__. There are also on average __8235__ streamers live playing __100__ games each. That makes sense since the data extracted was the top __100__ streamers for the top __100__ games. Lastly, there are __56702__ duplicate entries and there are __408913__ entries with unspecified `is_mature` filter and __399__ with unspecified `title`.
<hr>

#### 2.2.2 Most watched channels
<a id="most_watched"></a>

In [38]:
most_watched_df.head(10)

,Unnamed: 0,Unnamed: 1,Channel,Watch time,Stream time,Peak viewers,Average viewers,Followers,Followers gained,Views gained,Partnered,Mature,Language,Unnamed: 13
0,https://static-cdn.jtvnw.net/jtv_user_pictures...,501,twlevewinsHS,7850130,3165,4100,2480,63634,335,73164,True,False,English,twlevewinsHS
1,https://static-cdn.jtvnw.net/jtv_user_pictures...,502,scumtk,7837020,10080,1933,777,99856,-5817,74085,True,True,English,scumtk
2,https://static-cdn.jtvnw.net/jtv_user_pictures...,503,RTP Arena CS:GO (rtparenacsgo),7817445,2985,8849,2618,155033,2219,520960,True,False,Portuguese,RTP Arena CS:GO (rtparenacsgo)
3,https://static-cdn.jtvnw.net/jtv_user_pictures...,504,Kitboga,7790715,1005,12330,7751,929889,265,123758,True,False,English,Kitboga
4,https://static-cdn.jtvnw.net/jtv_user_pictures...,505,TpaBoMaH,7777800,2040,5770,3812,208323,1740,391756,True,False,Russian,TpaBoMaH
5,https://static-cdn.jtvnw.net/jtv_user_pictures...,506,pimpimenta,7776240,3705,5948,2098,873908,-66,300018,True,False,Portuguese,pimpimenta
6,https://static-cdn.jtvnw.net/jtv_user_pictures...,507,Flight23white,7774365,510,19516,15243,1545964,10069,126938,True,False,English,Flight23white
7,https://static-cdn.jtvnw.net/jtv_user_pictures...,508,NiteNightKid,7747095,2235,7552,3466,35657,214,72005,True,True,English,NiteNightKid
8,https://static-cdn.jtvnw.net/jtv_user_pictures...,509,포셔 (portialyn),7708500,2880,3973,2676,64712,175,114955,True,False,Korean,포셔 (portialyn)
9,https://static-cdn.jtvnw.net/jtv_user_pictures...,510,Pieface23,7701840,1275,11277,6040,480797,3180,88327,True,False,English,Pieface23


In [39]:
most_watched_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1000 entries, 0 to 99
Data columns (total 14 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Unnamed: 0        1000 non-null   object
 1   Unnamed: 1        1000 non-null   int64 
 2   Channel           1000 non-null   object
 3   Watch time        1000 non-null   int64 
 4   Stream time       1000 non-null   int64 
 5   Peak viewers      1000 non-null   int64 
 6   Average viewers   1000 non-null   int64 
 7   Followers         1000 non-null   int64 
 8   Followers gained  1000 non-null   int64 
 9   Views gained      1000 non-null   int64 
 10  Partnered         1000 non-null   bool  
 11  Mature            1000 non-null   bool  
 12  Language          1000 non-null   object
 13  Unnamed: 13       1000 non-null   object
dtypes: bool(2), int64(8), object(4)
memory usage: 103.5+ KB


What we see here are 3 `Unnamed:` columns that we should look into what they are and fix them.

In [40]:
most_watched_df.head(10)

,Unnamed: 0,Unnamed: 1,Channel,Watch time,Stream time,Peak viewers,Average viewers,Followers,Followers gained,Views gained,Partnered,Mature,Language,Unnamed: 13
0,https://static-cdn.jtvnw.net/jtv_user_pictures...,501,twlevewinsHS,7850130,3165,4100,2480,63634,335,73164,True,False,English,twlevewinsHS
1,https://static-cdn.jtvnw.net/jtv_user_pictures...,502,scumtk,7837020,10080,1933,777,99856,-5817,74085,True,True,English,scumtk
2,https://static-cdn.jtvnw.net/jtv_user_pictures...,503,RTP Arena CS:GO (rtparenacsgo),7817445,2985,8849,2618,155033,2219,520960,True,False,Portuguese,RTP Arena CS:GO (rtparenacsgo)
3,https://static-cdn.jtvnw.net/jtv_user_pictures...,504,Kitboga,7790715,1005,12330,7751,929889,265,123758,True,False,English,Kitboga
4,https://static-cdn.jtvnw.net/jtv_user_pictures...,505,TpaBoMaH,7777800,2040,5770,3812,208323,1740,391756,True,False,Russian,TpaBoMaH
5,https://static-cdn.jtvnw.net/jtv_user_pictures...,506,pimpimenta,7776240,3705,5948,2098,873908,-66,300018,True,False,Portuguese,pimpimenta
6,https://static-cdn.jtvnw.net/jtv_user_pictures...,507,Flight23white,7774365,510,19516,15243,1545964,10069,126938,True,False,English,Flight23white
7,https://static-cdn.jtvnw.net/jtv_user_pictures...,508,NiteNightKid,7747095,2235,7552,3466,35657,214,72005,True,True,English,NiteNightKid
8,https://static-cdn.jtvnw.net/jtv_user_pictures...,509,포셔 (portialyn),7708500,2880,3973,2676,64712,175,114955,True,False,Korean,포셔 (portialyn)
9,https://static-cdn.jtvnw.net/jtv_user_pictures...,510,Pieface23,7701840,1275,11277,6040,480797,3180,88327,True,False,English,Pieface23


`Channel` and `Unnamed: 13` look identical
<br>
`Unnamed: 1` looks like the standing
<br>
`Unnamed: 0` looks like a image CDN link
<br>
With those observations we can freely use the _pandas_ `drop` method to remove the unnecessary `Unnamed: 0` and `Unnamed: 13`. We are also going to place the standing as the index after sorting it. 

In [41]:
most_watched_df.drop(['Unnamed: 0', 'Unnamed: 13'], axis=1, inplace=True)

In [42]:
most_watched_df = most_watched_df.rename(columns={'Unnamed: 1': 'standing'})\
                                 .sort_values('standing', ascending=True)\
                                 .set_index('standing')

In [43]:
most_watched_df.head(10)

,Channel,Watch time,Stream time,Peak viewers,Average viewers,Followers,Followers gained,Views gained,Partnered,Mature,Language
standing,,,,,,,,,,,
1,xQcOW,572639745,5730,166377,99937,5522156,-2576985,7356085,True,False,English
2,RanbooLive,236614410,2865,149913,82587,2448959,125620,3045665,True,False,English
3,Gaules,235006260,10065,139944,23348,2656500,7778,3607539,True,True,Portuguese
4,loud_coringa,181608390,2790,121895,65092,1681866,57286,1118616,True,False,Portuguese
5,BLASTPremier,178571895,3405,134091,52444,948580,94180,5611173,True,False,English
6,summit1g,167065860,4095,88265,40797,5886358,11240,2607794,True,False,English
7,ludwig,160337520,3585,150360,44724,2673488,75444,3862949,True,False,English
8,RocketLeague,153100380,2400,116292,63791,3006488,42300,2337562,True,False,English
9,auronplay,146081295,1170,216805,124855,8192827,82808,2815563,True,False,Spanish


For ease of use and standard we are also going to _lowercase_ and remove all _whitespace_ from the columns names.

In [44]:
most_watched_df.columns = most_watched_df.columns.map(lambda x: x.lower().strip().replace(' ', '_'))

In [45]:
most_watched_df.head(10)

,channel,watch_time,stream_time,peak_viewers,average_viewers,followers,followers_gained,views_gained,partnered,mature,language
standing,,,,,,,,,,,
1,xQcOW,572639745,5730,166377,99937,5522156,-2576985,7356085,True,False,English
2,RanbooLive,236614410,2865,149913,82587,2448959,125620,3045665,True,False,English
3,Gaules,235006260,10065,139944,23348,2656500,7778,3607539,True,True,Portuguese
4,loud_coringa,181608390,2790,121895,65092,1681866,57286,1118616,True,False,Portuguese
5,BLASTPremier,178571895,3405,134091,52444,948580,94180,5611173,True,False,English
6,summit1g,167065860,4095,88265,40797,5886358,11240,2607794,True,False,English
7,ludwig,160337520,3585,150360,44724,2673488,75444,3862949,True,False,English
8,RocketLeague,153100380,2400,116292,63791,3006488,42300,2337562,True,False,English
9,auronplay,146081295,1170,216805,124855,8192827,82808,2815563,True,False,Spanish


In [46]:
most_watched_df.duplicated().sum()

0

In [47]:
most_watched_df.isnull().sum()

channel             0
watch_time          0
stream_time         0
peak_viewers        0
average_viewers     0
followers           0
followers_gained    0
views_gained        0
partnered           0
mature              0
language            0
dtype: int64

In [48]:
most_watched_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1000 entries, 1 to 1000
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   channel           1000 non-null   object
 1   watch_time        1000 non-null   int64 
 2   stream_time       1000 non-null   int64 
 3   peak_viewers      1000 non-null   int64 
 4   average_viewers   1000 non-null   int64 
 5   followers         1000 non-null   int64 
 6   followers_gained  1000 non-null   int64 
 7   views_gained      1000 non-null   int64 
 8   partnered         1000 non-null   bool  
 9   mature            1000 non-null   bool  
 10  language          1000 non-null   object
dtypes: bool(2), int64(7), object(2)
memory usage: 80.1+ KB


In [49]:
most_watched_df.describe()

,watch_time,stream_time,peak_viewers,average_viewers,followers,followers_gained,views_gained
count,1.000000e+03,1000.000000,1000.000000,1000.000000,1.000000e+03,1.000000e+03,1.000000e+03
mean,1.503488e+07,2706.765000,17128.013000,8319.368000,6.916881e+05,-3.814654e+03,4.262630e+05
std,2.800675e+07,1861.823406,28052.195577,16188.643371,1.166406e+06,1.346496e+05,6.995785e+05
min,4.225005e+06,75.000000,773.000000,431.000000,7.500000e+01,-2.576985e+06,2.667900e+04
25%,5.479042e+06,1590.000000,4805.250000,2222.500000,1.709855e+05,-1.280250e+03,1.329418e+05
50%,7.851428e+06,2332.500000,8432.000000,3840.500000,3.368890e+05,1.554500e+03,2.392250e+05
75%,1.445595e+07,3232.500000,16103.500000,7725.750000,7.196735e+05,7.096500e+03,4.554330e+05
max,5.726397e+08,10080.000000,359531.000000,230361.000000,1.667215e+07,1.595960e+05,1.150208e+07


After some manipulation, this dataset has __1000__ rows and __11__ columns.
<hr>

####  2.2.3 Fastest growing channels

In [50]:
fastest_growing_df.head(10)

,Unnamed: 0,Unnamed: 1,Channel,Watch time,Stream time,Peak viewers,Average viewers,Followers,Followers gained,Views gained,Partnered,Mature,Language,Unnamed: 13
0,https://static-cdn.jtvnw.net/jtv_user_pictures...,401,zin0xfnr,1605,495,9,3,22677,22617,191,False,False,German,zin0xfnr
1,https://static-cdn.jtvnw.net/jtv_user_pictures...,402,GreenAppleTiVi,16185,945,25,17,22986,22595,307,False,False,German,GreenAppleTiVi
2,https://static-cdn.jtvnw.net/jtv_user_pictures...,403,LaWhooligan,36090,810,142,44,23865,22592,2125,False,False,English,LaWhooligan
3,https://static-cdn.jtvnw.net/jtv_user_pictures...,404,V_aoen,8220,885,32,9,23137,22508,307,False,True,English,V_aoen
4,https://static-cdn.jtvnw.net/jtv_user_pictures...,405,zuzk4_aa,2310,450,13,5,22599,22505,121,False,False,Polish,zuzk4_aa
5,https://static-cdn.jtvnw.net/jtv_user_pictures...,406,PapoMC,16622640,1770,25966,9391,1621415,22467,698146,True,True,Spanish,PapoMC
6,https://static-cdn.jtvnw.net/jtv_user_pictures...,407,airestereo,11115,1320,105,8,41468,22387,294,False,False,English,airestereo
7,https://static-cdn.jtvnw.net/jtv_user_pictures...,408,gewoonroy1,8250,1335,18,6,22454,22368,365,False,False,Dutch,gewoonroy1
8,https://static-cdn.jtvnw.net/jtv_user_pictures...,409,elisawavess,3116235,2655,4750,1173,40417,22342,209876,True,True,Spanish,elisawavess
9,https://static-cdn.jtvnw.net/jtv_user_pictures...,410,Czech_KappeR,71955,1710,159,42,32995,22320,7645,False,False,Czech,Czech_KappeR


This dataset is going to receive the same treatment as the one preceding it.

In [51]:
fastest_growing_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1000 entries, 0 to 99
Data columns (total 14 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Unnamed: 0        1000 non-null   object
 1   Unnamed: 1        1000 non-null   int64 
 2   Channel           1000 non-null   object
 3   Watch time        1000 non-null   int64 
 4   Stream time       1000 non-null   int64 
 5   Peak viewers      1000 non-null   int64 
 6   Average viewers   1000 non-null   int64 
 7   Followers         1000 non-null   int64 
 8   Followers gained  1000 non-null   int64 
 9   Views gained      1000 non-null   int64 
 10  Partnered         1000 non-null   bool  
 11  Mature            1000 non-null   bool  
 12  Language          1000 non-null   object
 13  Unnamed: 13       1000 non-null   object
dtypes: bool(2), int64(8), object(4)
memory usage: 103.5+ KB


In [52]:
fastest_growing_df.drop(['Unnamed: 0', 'Unnamed: 13'], axis=1, inplace=True)

In [53]:
fastest_growing_df = fastest_growing_df.rename(columns={'Unnamed: 1': 'standing'})\
                                 .sort_values('standing', ascending=True)\
                                 .set_index('standing')

In [54]:
fastest_growing_df.columns = fastest_growing_df.columns.map(lambda x: x.lower().strip().replace(' ', '_'))

In [55]:
fastest_growing_df.duplicated().sum()

0

In [56]:
fastest_growing_df.isnull().sum()

channel             0
watch_time          0
stream_time         0
peak_viewers        0
average_viewers     0
followers           0
followers_gained    0
views_gained        0
partnered           0
mature              0
language            0
dtype: int64

In [57]:
fastest_growing_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1000 entries, 1 to 1000
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   channel           1000 non-null   object
 1   watch_time        1000 non-null   int64 
 2   stream_time       1000 non-null   int64 
 3   peak_viewers      1000 non-null   int64 
 4   average_viewers   1000 non-null   int64 
 5   followers         1000 non-null   int64 
 6   followers_gained  1000 non-null   int64 
 7   views_gained      1000 non-null   int64 
 8   partnered         1000 non-null   bool  
 9   mature            1000 non-null   bool  
 10  language          1000 non-null   object
dtypes: bool(2), int64(7), object(2)
memory usage: 80.1+ KB


In [58]:
fastest_growing_df.describe()

,watch_time,stream_time,peak_viewers,average_viewers,followers,followers_gained,views_gained
count,1.000000e+03,1000.000000,1000.000000,1000.000000,1.000000e+03,1.000000e+03,1.000000e+03
mean,4.934496e+06,1249.545000,7131.379000,3493.195000,2.543559e+05,2.948236e+04,1.432722e+05
std,1.886237e+07,1232.374767,25149.183835,14178.205184,8.048404e+05,4.969171e+04,5.034751e+05
min,6.000000e+01,30.000000,2.000000,1.000000,1.660000e+02,1.130700e+04,0.000000e+00
25%,2.943750e+03,371.250000,12.750000,4.000000,1.686725e+04,1.416825e+04,1.470000e+02
50%,2.982750e+04,937.500000,72.500000,30.000000,3.170700e+04,1.941450e+04,1.218000e+03
75%,6.743550e+05,1770.000000,1570.500000,653.250000,9.484775e+04,3.248100e+04,3.351125e+04
max,2.366144e+08,10080.000000,359531.000000,230361.000000,8.831874e+06,1.398095e+06,7.712756e+06


This dataset is familiar to the <a href="#most_watched">Most Watched Channels</a> consisting of __1000__ rows and __11__ columns.
<hr>

#### 2.2.4 Most watched games

In [59]:
games_df.head(10)

,Unnamed: 0,Unnamed: 1,Game,Watch time,Stream time,Peak viewers,Peak channels,Streamers,Average viewers,Average channels,Average viewer ratio,Followers gained,Views gained,FPR,VPR,Unnamed: 15
0,Grand Theft Auto V,1,Grand Theft Auto V,3858308160,36281700,838606,7043,81802,382768,3599,106.34,4050979,63497073,6.699210,105.006777,Grand Theft Auto V
1,Just Chatting,2,Just Chatting,3817617240,50377500,870817,8026,205830,378731,4997,75.78,5644892,99932398,6.723111,119.020275,Just Chatting
2,League of Legends,3,League of Legends,2038034640,51180600,529193,8333,117363,202185,5077,39.82,990206,44384031,1.160838,52.032252,League of Legends
3,Fortnite,4,Fortnite,1233289680,77842680,300531,13805,232235,122350,7722,15.84,6741075,34456004,5.195922,26.558184,Fortnite
4,VALORANT,5,VALORANT,1208156400,47779380,307693,7835,133072,119856,4740,25.29,1060664,33469807,1.331952,42.030441,VALORANT
5,Call of Duty: Warzone,6,Call of Duty: Warzone,1181475060,48264840,278820,8705,100493,117209,4788,24.48,818764,15754868,1.017839,19.585522,Call of Duty: Warzone
6,Minecraft,7,Minecraft,1073379300,43641660,341421,8305,154848,106486,4329,24.60,4028982,16635780,5.539178,22.871421,Minecraft
7,Counter-Strike: Global Offensive,8,Counter-Strike: Global Offensive,1036974120,20340180,362748,4016,62679,102874,2017,50.98,1310455,29275087,3.865615,86.356425,Counter-Strike: Global Offensive
8,Apex Legends,9,Apex Legends,709455180,52156980,120481,8110,126934,70382,5174,13.60,71790,12905807,0.082585,14.846496,Apex Legends
9,Dota 2,10,Dota 2,635300280,9344040,210268,1559,14851,63025,926,67.99,32446,13970781,0.208342,89.709254,Dota 2


This `DataFrame` also looks familiar although the `Unnamed:` columns include numbers __0, 1__ and __15__. `Unnamed: 0` and `Unnamed: 15` are the name of the game, and `Unnamed: 1` is the _standing_. We also have 2 abbreviate columns - `FPR` and `VPR`. `FPR` is the _follower\_per\_hour_ metric and `VPR` is the _viewers\_per\_hour_ metric. So we will be giving a similar treatment to this dataset as the previous 2.

In [60]:
games_df.drop(['Unnamed: 0', 'Unnamed: 15'], axis=1, inplace=True)

In [61]:
games_df = games_df.rename(columns={'Unnamed: 1': 'standing', 'FPR': 'hourly_following', 'VPR': 'hourly_viewing'})\
                                 .sort_values('standing', ascending=True)\
                                 .set_index('standing')

In [62]:
games_df.columns = games_df.columns.map(lambda x: x.lower().strip().replace(' ', '_'))

In [63]:
games_df.head(10)

,game,watch_time,stream_time,peak_viewers,peak_channels,streamers,average_viewers,average_channels,average_viewer_ratio,followers_gained,views_gained,hourly_following,hourly_viewing
standing,,,,,,,,,,,,,
1,Grand Theft Auto V,3858308160,36281700,838606,7043,81802,382768,3599,106.34,4050979,63497073,6.699210,105.006777
2,Just Chatting,3817617240,50377500,870817,8026,205830,378731,4997,75.78,5644892,99932398,6.723111,119.020275
3,League of Legends,2038034640,51180600,529193,8333,117363,202185,5077,39.82,990206,44384031,1.160838,52.032252
4,Fortnite,1233289680,77842680,300531,13805,232235,122350,7722,15.84,6741075,34456004,5.195922,26.558184
5,VALORANT,1208156400,47779380,307693,7835,133072,119856,4740,25.29,1060664,33469807,1.331952,42.030441
6,Call of Duty: Warzone,1181475060,48264840,278820,8705,100493,117209,4788,24.48,818764,15754868,1.017839,19.585522
7,Minecraft,1073379300,43641660,341421,8305,154848,106486,4329,24.60,4028982,16635780,5.539178,22.871421
8,Counter-Strike: Global Offensive,1036974120,20340180,362748,4016,62679,102874,2017,50.98,1310455,29275087,3.865615,86.356425
9,Apex Legends,709455180,52156980,120481,8110,126934,70382,5174,13.60,71790,12905807,0.082585,14.846496


In [64]:
games_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100 entries, 1 to 100
Data columns (total 13 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   game                  100 non-null    object 
 1   watch_time            100 non-null    int64  
 2   stream_time           100 non-null    int64  
 3   peak_viewers          100 non-null    int64  
 4   peak_channels         100 non-null    int64  
 5   streamers             100 non-null    int64  
 6   average_viewers       100 non-null    int64  
 7   average_channels      100 non-null    int64  
 8   average_viewer_ratio  100 non-null    float64
 9   followers_gained      100 non-null    int64  
 10  views_gained          100 non-null    int64  
 11  hourly_following      100 non-null    float64
 12  hourly_viewing        100 non-null    float64
dtypes: float64(3), int64(9), object(1)
memory usage: 10.9+ KB


In [65]:
games_df.describe()

,watch_time,stream_time,peak_viewers,peak_channels,streamers,average_viewers,average_channels,average_viewer_ratio,followers_gained,views_gained,hourly_following,hourly_viewing
count,1.000000e+02,1.000000e+02,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,1.000000e+02,1.000000e+02,100.000000,100.000000
mean,2.672602e+08,8.461568e+06,95778.110000,1639.630000,23557.510000,26513.390000,838.920000,50.236900,3.311183e+05,5.768350e+06,2.105594,74.772249
std,6.036091e+08,1.491981e+07,142943.920589,2628.929976,42703.520788,59881.807258,1480.144797,56.218514,1.036808e+06,1.341725e+07,3.337962,107.582508
min,2.845698e+07,1.407000e+05,8273.000000,47.000000,377.000000,2823.000000,13.000000,2.500000,-1.473900e+04,2.307030e+05,-1.751038,1.557201
25%,4.179219e+07,1.324635e+06,22320.750000,267.250000,3097.250000,4145.500000,131.000000,18.805000,1.531650e+04,8.500868e+05,0.364334,18.967798
50%,7.406493e+07,2.745180e+06,46124.500000,525.000000,6906.500000,7347.000000,271.500000,29.500000,3.933050e+04,1.585626e+06,0.916847,39.724988
75%,2.038080e+08,7.738185e+06,97953.750000,1608.500000,19656.000000,20218.500000,767.500000,60.337500,1.267705e+05,4.089154e+06,2.186640,83.555540
max,3.858308e+09,7.784268e+07,870817.000000,13805.000000,232235.000000,382768.000000,7722.000000,396.320000,6.741075e+06,9.993240e+07,20.899048,654.963300


After some manipulation, this dataset consists ot __100__ rows and __13__ columns.
<hr>

### 2.3 Data cleaning and visualization

Now we can wrangle and manipulate the data in a way to extract insights and visualize them. This process will help us later with the _feature engineering_ step. We are primarily looking for positive or at least moderate correlation.
<hr>

First step of the process is thoroughly cleaning and preparing the data for visualization. This will be done by standartizing the column names, removing the meaningless (`null` and/or `duplicates`) columns and converting the categorical columns into their numeric counterparts.

In [66]:
import pickle

from matplotlib import pyplot as plt
import seaborn as sns
sns.set_theme(style="darkgrid")

In [67]:
live_df['started_at'] = pd.to_datetime(live_df['started_at'], infer_datetime_format=True)
live_df['started_at']= live_df['started_at'].dt.strftime('%Y-%m-%d-%H:%M')
live_df = live_df.rename(columns={'is_mature': 'mature'})
live_df = live_df.reset_index()
live_df = live_df.dropna()
live_df = live_df.drop_duplicates()
live_df['mature'] = live_df['mature'].astype(float)
live_df = live_df.drop(['game_id', 'type', 'user_id', 'user_login'], axis=1)
live_df.head(10)

,date,language,started_at,title,user_name,viewer_count,game_name,mature
400855,04-15-19:08,zh,2021-04-15-04:29,【 HaHa 】 0415 Change Life 有人要來一起街友?,我是哈哈唷,12,Hobo: Tough Life,1.0
400885,04-15-19:08,en,2021-04-15-16:42,Survival in a different setting.,malaika2610,2,Hobo: Tough Life,1.0
400886,04-15-19:08,en,2021-04-15-16:31,Hobo Life for me - First day playing,BabySkyDiverTV,2,Hobo: Tough Life,1.0
400887,04-15-19:08,en,2021-04-15-16:48,Plus Ultra Day 1 | Gaming with the gang,DukeGoldwawe,3,Hobo: Tough Life,0.0
400888,04-15-19:08,cs,2021-04-15-12:14,"Prvotřídní život bezdomovce, podpora českých v...",Tvarooh,3,Hobo: Tough Life,0.0
400889,04-15-19:08,en,2021-04-15-16:03,From Rags To Riches,RascalKG,4,Hobo: Tough Life,0.0
400890,04-15-19:08,cs,2021-04-15-12:00,HOBO - Though Life - díky za každý follow!,Rowan_cz,4,Hobo: Tough Life,1.0
400891,04-15-19:08,en,2021-04-15-14:59,Living the Life of a Hobo,danath,4,Hobo: Tough Life,1.0
400892,04-15-19:08,pl,2021-04-15-16:14,Symulator menela\n,wonsiu12,4,Hobo: Tough Life,0.0
400893,04-15-19:08,cs,2021-04-15-15:57,Bezdomovecké kočkoholky.,xXmonsterkaXx,5,Hobo: Tough Life,0.0


In [68]:
with open('./pickles/iso_codes.pkl', 'rb') as f:
    iso_639_choices = pickle.load(f)

In [69]:
most_watched_df = most_watched_df.reset_index()
most_watched_df['language'] = most_watched_df['language'].map(iso_639_choices)
most_watched_df['partnered'], most_watched_df['mature'] = most_watched_df['partnered'].astype(float), most_watched_df['mature'].astype(float)
most_watched_df = most_watched_df.dropna()
most_watched_df = most_watched_df.drop_duplicates()
most_watched_df.head(10)

,standing,channel,watch_time,stream_time,peak_viewers,average_viewers,followers,followers_gained,views_gained,partnered,mature,language
0,1,xQcOW,572639745,5730,166377,99937,5522156,-2576985,7356085,1.0,0.0,en
1,2,RanbooLive,236614410,2865,149913,82587,2448959,125620,3045665,1.0,0.0,en
2,3,Gaules,235006260,10065,139944,23348,2656500,7778,3607539,1.0,1.0,pt
3,4,loud_coringa,181608390,2790,121895,65092,1681866,57286,1118616,1.0,0.0,pt
4,5,BLASTPremier,178571895,3405,134091,52444,948580,94180,5611173,1.0,0.0,en
5,6,summit1g,167065860,4095,88265,40797,5886358,11240,2607794,1.0,0.0,en
6,7,ludwig,160337520,3585,150360,44724,2673488,75444,3862949,1.0,0.0,en
7,8,RocketLeague,153100380,2400,116292,63791,3006488,42300,2337562,1.0,0.0,en
9,10,Rainbow6,140292660,3585,77427,39133,1869263,25309,2225554,1.0,1.0,en
11,12,loltyler1,134579355,3990,68350,33729,4246342,10016,2294842,1.0,0.0,en


In [70]:
fastest_growing_df = fastest_growing_df.reset_index()
fastest_growing_df['language'] = fastest_growing_df['language'].map(iso_639_choices)
fastest_growing_df['partnered'], fastest_growing_df['mature'] = fastest_growing_df['partnered'].astype(float), fastest_growing_df['mature'].astype(float)
fastest_growing_df = fastest_growing_df.dropna()
fastest_growing_df = fastest_growing_df.drop_duplicates()
fastest_growing_df.head(10)

,standing,channel,watch_time,stream_time,peak_viewers,average_viewers,followers,followers_gained,views_gained,partnered,mature,language
0,1,NeonSniperPanda,5040,135,65,37,1602392,1398095,351,0.0,0.0,en
1,2,klairelise,120990,825,233,146,372545,345174,13430,1.0,1.0,en
2,3,jessjessjessu,3272250,555,12801,5895,334977,165157,261392,0.0,0.0,en
3,4,TubboLIVE,68963790,2130,63230,32377,934471,159596,1227726,1.0,0.0,en
4,5,LouiKey,450,315,2,1,200557,157141,25,0.0,0.0,en
5,6,ElisabeteTV,3250260,3090,2054,1051,301993,152603,157644,0.0,0.0,en
7,8,Indiefoxx,23449620,2055,17509,11411,738426,143615,2682257,1.0,0.0,en
8,9,djscheme,94530,330,472,286,172734,135477,2378,1.0,0.0,en
9,10,noah_zek,585,270,3,2,133351,132624,109,0.0,1.0,en
10,11,Otto,60915,900,110,67,167187,129166,2772,1.0,0.0,en


In [71]:
games_df = games_df.reset_index()
games_df = games_df.dropna()
games_df = games_df.drop_duplicates()
games_df.head(10)

,standing,game,watch_time,stream_time,peak_viewers,peak_channels,streamers,average_viewers,average_channels,average_viewer_ratio,followers_gained,views_gained,hourly_following,hourly_viewing
0,1,Grand Theft Auto V,3858308160,36281700,838606,7043,81802,382768,3599,106.34,4050979,63497073,6.699210,105.006777
1,2,Just Chatting,3817617240,50377500,870817,8026,205830,378731,4997,75.78,5644892,99932398,6.723111,119.020275
2,3,League of Legends,2038034640,51180600,529193,8333,117363,202185,5077,39.82,990206,44384031,1.160838,52.032252
3,4,Fortnite,1233289680,77842680,300531,13805,232235,122350,7722,15.84,6741075,34456004,5.195922,26.558184
4,5,VALORANT,1208156400,47779380,307693,7835,133072,119856,4740,25.29,1060664,33469807,1.331952,42.030441
5,6,Call of Duty: Warzone,1181475060,48264840,278820,8705,100493,117209,4788,24.48,818764,15754868,1.017839,19.585522
6,7,Minecraft,1073379300,43641660,341421,8305,154848,106486,4329,24.60,4028982,16635780,5.539178,22.871421
7,8,Counter-Strike: Global Offensive,1036974120,20340180,362748,4016,62679,102874,2017,50.98,1310455,29275087,3.865615,86.356425
8,9,Apex Legends,709455180,52156980,120481,8110,126934,70382,5174,13.60,71790,12905807,0.082585,14.846496
9,10,Dota 2,635300280,9344040,210268,1559,14851,63025,926,67.99,32446,13970781,0.208342,89.709254


Now that we have that done, we can combining datasets. The first two datasets we are going to combine are the `fastest_growing` and `most_watched`. They show us the possible top streamers for that week whose success lies within averaging a lot of viewers, gaining viewers or attracting following. We will we calling that combined dataset `mp` standing for __most performing__.

In [72]:
mp_df = pd.concat([fastest_growing_df, most_watched_df], axis=0)
mp_df = mp_df.set_index('channel')
mp_df = mp_df[~mp_df.index.duplicated(keep='first')]
mp_df = mp_df.drop(['standing'], axis=1)
mp_df = mp_df.reset_index()

In [73]:
len(mp_df)

1620

In [74]:
mp_df['followers_gained'].max()

1398095

In [75]:
mp_df['followers_gained'].min()

-2576985

In [76]:
mp_df.loc[mp_df['followers_gained'] == mp_df['followers_gained'].max()]

,channel,watch_time,stream_time,peak_viewers,average_viewers,followers,followers_gained,views_gained,partnered,mature,language
0,NeonSniperPanda,5040,135,65,37,1602392,1398095,351,0.0,0.0,en


In [77]:
mp_df.loc[mp_df['followers_gained'] == mp_df['followers_gained'].min()]

,channel,watch_time,stream_time,peak_viewers,average_viewers,followers,followers_gained,views_gained,partnered,mature,language
841,xQcOW,572639745,5730,166377,99937,5522156,-2576985,7356085,1.0,0.0,en


In [78]:
mp_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1620 entries, 0 to 1619
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   channel           1620 non-null   object 
 1   watch_time        1620 non-null   int64  
 2   stream_time       1620 non-null   int64  
 3   peak_viewers      1620 non-null   int64  
 4   average_viewers   1620 non-null   int64  
 5   followers         1620 non-null   int64  
 6   followers_gained  1620 non-null   int64  
 7   views_gained      1620 non-null   int64  
 8   partnered         1620 non-null   float64
 9   mature            1620 non-null   float64
 10  language          1620 non-null   object 
dtypes: float64(2), int64(7), object(2)
memory usage: 139.3+ KB


In [79]:
mp_df.describe()

,watch_time,stream_time,peak_viewers,average_viewers,followers,followers_gained,views_gained,partnered,mature
count,1.620000e+03,1620.000000,1620.000000,1620.000000,1.620000e+03,1.620000e+03,1.620000e+03,1620.000000,1620.000000
mean,8.301254e+06,2000.842593,9666.649383,4679.035802,3.862062e+05,9.240171e+03,2.299733e+05,0.596296,0.236420
std,2.222596e+07,1802.341560,22485.343035,12742.559823,8.845360e+05,1.132897e+05,5.506302e+05,0.490791,0.425014
min,7.500000e+01,30.000000,2.000000,1.000000,7.500000e+01,-2.576985e+06,0.000000e+00,0.000000,0.000000
25%,2.197125e+04,735.000000,53.000000,23.000000,2.728050e+04,8.310000e+02,8.360000e+02,0.000000,0.000000
50%,4.652708e+06,1665.000000,3406.500000,1624.000000,1.152510e+05,1.160350e+04,9.149350e+04,1.000000,0.000000
75%,8.694056e+06,2733.750000,9442.000000,4465.000000,3.725698e+05,1.969850e+04,2.560472e+05,1.000000,0.000000
max,5.726397e+08,10080.000000,359531.000000,230361.000000,1.667215e+07,1.398095e+06,1.150208e+07,1.000000,1.000000


What we get from the `mp` dataset is that we have __1620__ benchmark streamers each with their respective milestones. We have also looked at one of the benchmarks for success, namely the `followers_gained` where we have some peculiarities that we will later look into while visualizing.
<hr>
Now we need to see which streamer which game played in the extracted span.

In [80]:
#for i in range(len(mp_df)):
    #mp_df.loc[mp_df.index[i], 'rate'] = 1 * (mp_df.iloc[i]['followers_gained'] / mp_df['followers_gained'].max())

Firstly, `user_name` is the equivalent of `channel` so we might as well rename it.

In [81]:
live_df = live_df.rename(columns={'user_name': 'channel'})

In [86]:
test = pd.concat([live_df, mp_df])

In [103]:
live_df[live_df['channel'] == 'NeonSniperPanda']

,date,language,started_at,title,channel,viewer_count,game_name,mature


In [97]:
games_df[games_df['game'] == 'Hobo: Tough Life']

,standing,game,watch_time,stream_time,peak_viewers,peak_channels,streamers,average_viewers,average_channels,average_viewer_ratio,followers_gained,views_gained,hourly_following,hourly_viewing
76,77,Hobo: Tough Life,40648860,191760,37671,50,551,4032,19,211.98,-1916,1013263,-0.599499,317.040989


In [96]:
test[test['game_name'] == 'Hobo: Tough Life']['watch_time'].median()

/home/mykull/miniconda3/envs/twitch-scrape/lib/python3.7/site-packages/numpy/lib/nanfunctions.py:1111: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)


nan

In [100]:
test[test['game_name'] == 'Fortnite']

,date,language,started_at,title,channel,viewer_count,game_name,mature,watch_time,stream_time,peak_viewers,average_viewers,followers,followers_gained,views_gained,partnered
411123,04-15-20:09,en,2021-04-15-15:32,need help !OPS,AsianJeff,394.0,Fortnite,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
411124,04-15-20:09,pt,2021-04-14-20:00,🔴 RERUN (SUBS 2x) LOJINHA TA ON FIRE FAMILIA |...,Faah,383.0,Fortnite,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
411125,04-15-20:09,en,2021-04-15-12:13,Viewing Party EU Dreamhack Cash Cup - !subrevi...,CoachBoop,340.0,Fortnite,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
411126,04-15-20:09,pt,2021-04-15-15:34,AQUECENDO PARA O CAMPEONATO DO LELEO FT. HRKZE...,Lessloko,330.0,Fortnite,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
411127,04-15-20:09,ar,2021-04-15-18:02,7000 نقطه حياكمم الله || SaC: NS,waleed2,322.0,Fortnite,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
411128,04-15-20:09,pt,2021-04-15-16:01,Bora fortnite? !youtube !insta 🔥 Code Manteig...,Manteiga,308.0,Fortnite,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
411129,04-15-20:09,fr,2021-04-15-16:43,DH extra double cheddar | Code créateur : Osl...,Oslo,299.0,Fortnite,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
411130,04-15-20:09,pt,2021-04-15-17:02,O TRIO PATRI E SETTI 🔵 ❗LOJA ❗SUB 🔵,MamuteLIVE,297.0,Fortnite,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
411131,04-15-20:09,es,2021-04-15-13:58,TRIO CUSTOMS w/ Pablito & Makro | !tw !code !v...,belvid,294.0,Fortnite,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
411132,04-15-20:09,es,2021-04-15-12:36,ROTATS CODE X !TROVO | USE CODE PALERKUMA #AD ...,Palermo,288.0,Fortnite,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Now we can combine the `mp` dataset together with the live data dataset (`live`).

In [88]:
combined_df = pd.merge(live_df, mp_df, left_on=['channel', 'language', 'mature'], right_on=['channel', 'language', 'mature'])

In [89]:
combined_df[combined_df['channel'] == 'ludwig']['game_name'].unique()

array(['Just Chatting', 'Chess', 'League of Legends'], dtype=object)

In [90]:
combined_df['game_name'].nunique()

223

In [91]:
combined_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8817 entries, 0 to 8816
Data columns (total 16 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   date              8817 non-null   object 
 1   language          8817 non-null   object 
 2   started_at        8817 non-null   object 
 3   title             8817 non-null   object 
 4   channel           8817 non-null   object 
 5   viewer_count      8817 non-null   int64  
 6   game_name         8817 non-null   object 
 7   mature            8817 non-null   float64
 8   watch_time        8817 non-null   int64  
 9   stream_time       8817 non-null   int64  
 10  peak_viewers      8817 non-null   int64  
 11  average_viewers   8817 non-null   int64  
 12  followers         8817 non-null   int64  
 13  followers_gained  8817 non-null   int64  
 14  views_gained      8817 non-null   int64  
 15  partnered         8817 non-null   float64
dtypes: float64(2), int64(8), object(6)
memory 

In [92]:
combined_df['date'].min()

'04-15-19:08'

In [93]:
combined_df['date'].max()

'04-17-09:06'

In [94]:
combined_df[combined_df['channel'] == 'Amouranth']

,date,language,started_at,title,channel,viewer_count,game_name,mature,watch_time,stream_time,peak_viewers,average_viewers,followers,followers_gained,views_gained,partnered
6412,04-16-02:12,en,2021-04-16-00:01,E-GIRL TO STUDY AND RELAX TO💦 !instagram: amo...,Amouranth,3744,Just Chatting,0.0,68325495,5565,21371,12277,2530559,76206,7712756,1.0
6413,04-16-03:13,en,2021-04-16-00:01,ASMR 💦 use headphones - relax with me! 💦 !inst...,Amouranth,7960,ASMR,0.0,68325495,5565,21371,12277,2530559,76206,7712756,1.0
6414,04-16-04:13,en,2021-04-16-00:01,ASMR 💦 use headphones - relax with me! 💦 !inst...,Amouranth,8172,ASMR,0.0,68325495,5565,21371,12277,2530559,76206,7712756,1.0
6415,04-16-05:14,en,2021-04-16-00:01,ASMR 💦 use headphones - relax with me! 💦 !inst...,Amouranth,8258,ASMR,0.0,68325495,5565,21371,12277,2530559,76206,7712756,1.0
6416,04-16-06:14,en,2021-04-16-00:01,ASMR 💦 use headphones - relax with me! 💦 !inst...,Amouranth,8236,ASMR,0.0,68325495,5565,21371,12277,2530559,76206,7712756,1.0
6417,04-16-07:15,en,2021-04-16-00:01,ASMR 💦 use headphones - relax with me! 💦 !inst...,Amouranth,9248,ASMR,0.0,68325495,5565,21371,12277,2530559,76206,7712756,1.0
6418,04-16-08:15,en,2021-04-16-00:01,ASMR 💦 use headphones - relax with me! 💦 !inst...,Amouranth,9842,ASMR,0.0,68325495,5565,21371,12277,2530559,76206,7712756,1.0
6419,04-16-09:16,en,2021-04-16-00:01,💦🍑HOT TUB🍑💦BIKINI TRY ON💦4K💦 !instagram: amour...,Amouranth,18715,Just Chatting,0.0,68325495,5565,21371,12277,2530559,76206,7712756,1.0
6420,04-16-10:16,en,2021-04-16-00:01,💦🍑HOT TUB🍑💦BIKINI TRY ON💦4K💦 !instagram: amour...,Amouranth,15087,Just Chatting,0.0,68325495,5565,21371,12277,2530559,76206,7712756,1.0
6421,04-16-11:17,en,2021-04-16-00:01,💦🍑HOT TUB🍑💦BIKINI TRY ON💦4K💦 !instagram: amour...,Amouranth,16272,Just Chatting,0.0,68325495,5565,21371,12277,2530559,76206,7712756,1.0


Now that we have combined all the streamer data, we can do the final combination between the streamer data and game data.

In [ ]:
df = pd.merge(combined_df, games_df, left_on='game_name', right_on='game', suffixes=['_channel', '_game'])

In [ ]:
df = df.drop(['standing', 'game'], axis=1)
df = df.rename(columns={'peak_channels': 'peak_channels_game', 
                            'average_channels': 'average_channels_game', 
                            'average_viewer_ratio': 'average_viewer_ratio_game', 
                            'hourly_following': 'hourly_following_game', 
                            'hourly_viewing': 'hourly_viewing_game', 
                            'streamers': 'weekly_streamers_game', 
                            'date': 'timestamp'})

In [ ]:
df.head(10)

Now that we have combined all the available data into the final `df` we can look for correlations and start visualizing patterns. We have saved some trouble as we have already converted the _categorical_ data to its _numeric_ counterpart.